# DATA20001 Deep Learning - Group Project
## Image project

- Testing [densenet201](https://pytorch.org/docs/stable/torchvision/models.html#id16) implementation of pretrained transfer learning models
- Data preprocessing includes data augmentation, shuffle and split train=17k, val=1.5k, test=1.5k
- Data augmentation IS applied
- No data balancing applied



In [7]:
# automatically reload dependencies and repository content so that kernel need not be restarted
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Import dependencies and select correct device

In [8]:
# import dependencies
import train_model
import eval_model
import utils
from image_dataset import ImageDataset
from data_augmentation import DataAugmentation

import time
import torch
from torchvision import models
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

In [9]:
device = None
if torch.cuda.is_available():
    print("Using GPU")
    device = torch.device("cuda:0")
else:
    print("Using CPU")
    device = torch.device("cpu")

Using GPU


## Define datasets

- shuffle
- train: 17k
- valid: 1.5k
- test: 1.5k

In [10]:
mapping_file = "file_to_labels_table.csv"
df = pd.read_csv(mapping_file)
train_idx, val_idx = utils.get_train_val_indexes(df, 0.85, shuffle=True)
val_test_split_idx = int(val_idx.shape[0]*.5)
test_idx = val_idx[:val_test_split_idx]
val_idx = val_idx[val_test_split_idx:]

dataAugmentation = DataAugmentation()
trainDataset = ImageDataset(train_idx, dataAugmentation=dataAugmentation)
valDataset = ImageDataset(val_idx)
testDataset = ImageDataset(test_idx)

print("Saving test dataset temporarily so test can be performed after restarting kernel.")
utils.save_pickle(testDataset, "tmp_test_data_exp_transfer_learning_6.pkl")
print("# Train:", len(trainDataset))
print("# Valid:", len(valDataset))
print("# Test:", len(testDataset))

Saving test dataset temporarily so test can be performed after restarting kernel.
# Train: 17000
# Valid: 1500
# Test: 1500


## Define the model architecture

The initial transfer learning implementation

In [11]:
torch.cuda.empty_cache()

preTrainedModel = models.densenet201(pretrained=True)

for param in preTrainedModel.parameters():
    param.requires_grad = False

num_ftrs = preTrainedModel.classifier.in_features
num_classes = 14

preTrainedModel.classifier = torch.nn.Sequential(
    torch.nn.Linear(num_ftrs, num_classes),
    torch.nn.Sigmoid()
)

model = preTrainedModel.to(device)

## Define test case

Hyperparam search could be defined here. No hyperparam searches in this notebook as purpose of this test is to test the effect of data preprocessing technique on initial transfer learning model architecture and see how longer run on epochs affect loss.

**Test objective**:

- Testing [densenet201](https://pytorch.org/docs/stable/torchvision/models.html#id16) implementation of pretrained transfer learning models
- Data preprocessing includes 
  - data augmentation, 
  - shuffle 
  - split train=17k, val=1.5k, test=1.5k


**Training results**:

- ...

In [12]:
n_epochs = 6

model, logs = train_model.train_model(
    model,
    trainDataset,
    valDataset,
    device,
    numberOfEpochs=n_epochs,
    returnLogs=True
)

Epoch 0/6
----------
Validating...

Progress: 100%

Loss: 0.7370 Accuracy: 0.4297
Precision: 0.0760 Recall: 0.6313
F1: 0.1357 Duration: 1m 8s

Epoch 1/6
----------
Training...

Progress: 100%

Loss: 0.1466 Accuracy: 0.9467
Precision: 0.7147 Recall: 0.4341
F1: 0.5401 Duration: 17m 46s

Validating...

Progress: 100%

Loss: 0.1177 Accuracy: 0.9556
Precision: 0.6633 Recall: 0.7596
F1: 0.7082 Duration: 1m 2s

Epoch 2/6
----------
Training...

Progress: 100%

Loss: 0.1334 Accuracy: 0.9512
Precision: 0.7278 Recall: 0.5179
F1: 0.6052 Duration: 13m 50s

Validating...

Progress: 100%

Loss: 0.1226 Accuracy: 0.9540
Precision: 0.6575 Recall: 0.7347
F1: 0.6939 Duration: 1m 8s

Epoch 3/6
----------
Training...

Progress: 100%

Loss: 0.1306 Accuracy: 0.9525
Precision: 0.7310 Recall: 0.5398
F1: 0.6210 Duration: 16m 10s

Validating...

Progress: 100%

Loss: 0.1233 Accuracy: 0.9544
Precision: 0.6736 Recall: 0.6917
F1: 0.6826 Duration: 1m 6s

Epoch 4/6
----------
Training...

Progress: 100%

Loss: 0.1316

## Save model and logs

It might be useful to save your model if you want to continue your work later, or use it for inference later.

In [19]:
pd.DataFrame(logs["val"]).to_csv("exp_transfer_learning_6_logs_val.csv", index=False)
pd.DataFrame(logs["train"]).to_csv("exp_transfer_learning_6_logs_train.csv", index=False)
torch.save(model, 'exp_transfer_learning_6_model.pkl')  # saves model with architecture

## Test model

- Load model back from state_dict and perform test on testing set
- Load test-set
- Load training time logs of performance

In [20]:
model = torch.load("exp_transfer_learning_6_model.pkl")
testDataset = utils.load_pickle("tmp_test_data_exp_transfer_learning_6.pkl")
logsTable = pd.read_csv("exp_transfer_learning_6_logs_val.csv")
y_hats, y_trues = eval_model.test_model(model, testDataset, device)

### Validation set performance during training

In [21]:
logsTable

,duration,loss,accuracy,precision,recall,f1,true positives,true negatives,false positives,false negatives
0,68.463634,0.736977,0.429667,0.076003,0.631296,0.135672,940,8083,11428,549
1,61.664144,0.117738,0.955619,0.663343,0.759570,0.708203,1131,18937,574,358
2,67.916258,0.122640,0.954048,0.657452,0.734721,0.693942,1094,18941,570,395
3,65.846471,0.123268,0.954381,0.673643,0.691739,0.682571,1030,19012,499,459
4,77.996974,0.127152,0.952381,0.634562,0.774345,0.697520,1153,18847,664,336
5,81.774200,0.110411,0.959714,0.701567,0.751511,0.725681,1119,19035,476,370
6,61.912110,0.108371,0.961286,0.728378,0.723976,0.726170,1078,19109,402,411


### Testing set performance

In [22]:
print("Testing set results.")
print(f" Accuracy\t{round(eval_model.get_metric(y_trues, y_hats, 'accuracy'), 3)}")
print(f" Precision\t{round(eval_model.get_metric(y_trues, y_hats, 'precision'), 3)}")
print(f" Recall \t{round(eval_model.get_metric(y_trues, y_hats, 'recall'), 3)}")
print(f" F1-score\t{round(eval_model.get_metric(y_trues, y_hats, 'f1'), 3)}")

Testing set results.
 Accuracy	0.957
 Precision	0.703
 Recall 	0.724
 F1-score	0.713


### Extra analysis

How many positives and negatives has been predicted.

Tuples like: **(label, count)**

In [23]:
list(zip(*np.unique(y_hats, return_counts=True)))

[(0, 19393), (1, 1607)]

## Download test set

The testset will be made available during the last week before the deadline and can be downloaded in the same way as the training set.

## Predict for test set

You should return your predictions for the test set in a plain text file.  The text file contains one row for each test set image.  Each row contains a binary prediction for each label (separated by a single space), 1 if it's present in the image, and 0 if not. The order of the labels is as follows (alphabetic order of the label names):

    baby bird car clouds dog female flower male night people portrait river sea tree

An example row could like like this if your system predicts the presense of a bird and clouds:

    0 1 0 1 0 0 0 0 0 0 0 0 0 0
    
The order of the rows should be according to the numeric order of the image numbers.  In the test set, this means that the first row refers to image `im20001.jpg`, the second to `im20002.jpg`, and so on.

If you have the prediction output matrix prepared in `y` you can use the following function to save it to a text file.

In [ ]:
np.savetxt('results.txt', y, fmt='%d')